In [ ]:
import os
from collections import defaultdict

import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Rectangle

import numpy as np
import pandas as pd

from tensorboard.backend.event_processing.event_accumulator import EventAccumulator

# Define a scaling factor
scale_factor = 1.5

# Update default font sizes by multiplying with the scaling factor
rcParams['font.size'] *= scale_factor       # Default font size

In [ ]:
training_seed = 1
test_seed = 101
env_id = "CARLPendulum"
method_names = {
    "stacked": "stacked",
}
test_env_config_ids = [f"test_{i}" for i in range(1, 25+1)]
c1_values = [0.1, 0.5, 1.0, 1.5, 2.0]
c2_values = [0.1, 0.5, 1.0, 1.5, 2.0]

In [ ]:
env_config_id = "default"

# checkpoint_idxs = defaultdict(list)
return_means = defaultdict(list)
return_stds = defaultdict(list)

for method in method_names:
    for test_env_config_id in test_env_config_ids:
        checkpoint_directory = f"../../runs/test/seed_{training_seed}/{env_id}/{test_env_config_id}/{test_env_config_id}/{method}/checkpoint_50000"
        if not os.path.exists(checkpoint_directory):
            return_means[method].append(1)
            return_stds[method].append(1)
            continue
        filenames = os.listdir(checkpoint_directory)
        for filename in filenames:
            if filename.startswith("event"):
                event_filename = filename
                break
        event_filename = os.path.join(checkpoint_directory, event_filename)

        event_acc = EventAccumulator(event_filename)
        event_acc.Reload()

        # print(event_acc.Tags())

        try:
            for e in event_acc.Scalars(f"evaluation/seed_{test_seed}/episodic_return_mean"):
                # checkpoint_idxs[method].append(e.step)
                return_means[method].append(e.value)

            for e in event_acc.Scalars(f"evaluation/seed_{test_seed}/episodic_return_std"):
                # checkpoint_idxs[method].append(e.step)
                return_stds[method].append(e.value)
        except:
            return_means[method].append(1)
            return_stds[method].append(1)
            continue

for method in return_means:
    fig, ax = plt.subplots(figsize=(10, 8))

    # Create the heatmap
    performance_data = np.array(return_means[method]).reshape((5, 5))
    cax = ax.matshow(performance_data, cmap='viridis', alpha=0.8, vmin=-1500, vmax=0)

    # Add color bar to show the scale
    plt.colorbar(cax)

    # Setting the tick marks
    ax.set_yticks(np.arange(len(c1_values)))
    ax.set_xticks(np.arange(len(c2_values)))
    # Set axis labels
    ax.set_yticklabels(list(c1_values))
    ax.set_xticklabels(list(c2_values))

    # Rotate the tick labels for the x-axis
    plt.xticks(rotation=0)

    # Set the title and axis labels
    ax.set_ylabel('length', labelpad=10, fontsize=18)
    ax.set_xlabel('mass', labelpad=10, fontsize=18)
    # ax.set_title(f'{method_names[method]}')
    ax.set_title(f'Near-Optimal')
    ax.xaxis.set_ticks_position('bottom')

    # Annotate each cell with the numeric value
    for i in range(len(c1_values)):
        for j in range(len(c2_values)):
            text = ax.text(j, i, int(performance_data[i, j]),
                        ha="center", va="center", color="black")

    os.makedirs(f"output/{env_id}", exist_ok=True)
    plt.savefig(f"output/{env_id}/near_optimal_{method}.pdf")
    plt.show()